In [ ]:
import numpy as np
import json
import time
import pickle
import sys
import nltk
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import unicodedata
import json
from pprint import pprint

In [ ]:
with open('train-v2.0.json') as f:
    data = json.load(f)

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
vocab=set()

In [ ]:
def preprocess(sentence):
    wnl = WordNetLemmatizer()
    temp = pos_tag(word_tokenize(sentence.lower()))
    y = []
    x = []
    count=0
    for i,j in temp:
        y.append(count)
        if i==u'``' or i==u"''":
            x.append('"')
            count+=1
        else:
            if j[0].lower() in ['a','n','v']:
                temp_i = wnl.lemmatize(i,j[0].lower())
                x.append(unicodedata.normalize('NFKD', temp_i).encode('ascii','ignore'))
            else:
                temp_i = wnl.lemmatize(i)
                x.append(unicodedata.normalize('NFKD', temp_i).encode('ascii','ignore'))
            count+=len(i)
        while count<len(sentence) and sentence[count]==' ':
            count+=1
        for k in range(len(x[-1])):
            vocab.add(x[-1][k])
    return x, y
def create_dataset(data):
    data_map = []
    errors = 0
    for i in data['data']:
        for j in i['paragraphs']:
            context,idx = preprocess(j['context'])
#             for x in range(len(idx)-1):
#                 print idx[x], j['context'][idx[x]:idx[x+1]]
#             print context
#             print idx
            for k in j['qas']:
                temp = {}
                qas,_ = preprocess(k['question'])
                temp['title']=i['title']
                temp['context']=context
                temp['question']=qas
                temp['is_impossible']=k['is_impossible']
                
                if k['is_impossible']:
                    if len(k['plausible_answers'])==0:
                        continue
                    ans_temp = k['plausible_answers'][0]
                else:
                    if len(k['answers'])==0:
                        continue
                    ans_temp = k['answers'][0]
                ans,_ = preprocess(ans_temp['text'])
                try:
                    temp['answer_start']=idx.index(ans_temp['answer_start'])
                    temp['answer_end']=idx.index(ans_temp['answer_start'])+len(ans)
                    temp['answer']=ans_temp['text']
                    data_map.append(temp)
                except:
                    errors+=1
                    temp['answer_start']=-1
                    temp['answer_end']=-1
                    temp['answer']=ans_temp['text']
                    data_map.append(temp)
    print "Errors="+str(errors)
    return data_map

In [ ]:
%%time
t = create_dataset(data)

In [ ]:
vocab

In [ ]:
d = {'data': t}
json.dump(d, open("train_data.json","w"))

In [ ]:
with open('dev-v2.0.json') as f:
    dev = json.load(f)

In [ ]:
%%time
test_d = create_dataset(dev)

In [ ]:
d = {'data': test_d}
json.dump(d, open("test_data.json","w"))

In [ ]:
with open("vocab.pkl","wb") as f:
    pickle.dump(list(vocab), f)